In [1]:
#Import librarires
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l2

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix

from mlxtend.plotting import plot_confusion_matrix

In [9]:
#Import dataset from online repo
#FIRST NEEDS - pip install ucimlrepo

from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
ionosphere = fetch_ucirepo(id=52) 
  
# data (as pandas dataframes) 
X = ionosphere.data.features 
y = ionosphere.data.targets 
  
# metadata 
print(ionosphere.metadata) 
  
# variable information 
print(ionosphere.variables) 

""" 
Sigillito,V., Wing,S., Hutton,L., and Baker,K.. (1989). Ionosphere. UCI Machine Learning Repository. https://doi.org/10.24432/C5W01B.
"""


{'uci_id': 52, 'name': 'Ionosphere', 'repository_url': 'https://archive.ics.uci.edu/dataset/52/ionosphere', 'data_url': 'https://archive.ics.uci.edu/static/public/52/data.csv', 'abstract': 'Classification of radar returns from the ionosphere', 'area': 'Physics and Chemistry', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 351, 'num_features': 34, 'feature_types': ['Integer', 'Real'], 'demographics': [], 'target_col': ['Class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1989, 'last_updated': 'Sun Jan 01 1989', 'dataset_doi': '10.24432/C5W01B', 'creators': ['V. Sigillito', 'S. Wing', 'L. Hutton', 'K. Baker'], 'intro_paper': None, 'additional_info': {'summary': 'This radar data was collected by a system in Goose Bay, Labrador.  This system consists of a phased array of 16 high-frequency antennas with a total transmitted power on the order of 6.4 kilowatts.  See the paper for more details.  

' \nSigillito,V., Wing,S., Hutton,L., and Baker,K.. (1989). Ionosphere. UCI Machine Learning Repository. https://doi.org/10.24432/C5W01B.\n'

In [12]:
features = X.to_numpy()
labels = y.to_numpy()

In [19]:
print(f"Features shape: {features.shape}")
print(f"Labels shape: {labels.shape}")


Features shape: (351, 34)
Labels shape: (351, 1)


In [22]:
#Split dataset into training, cv, test

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.4, random_state=35)

#Split test 50:50 for cv and test sets
X_test, X_cv, y_test, y_cv = train_test_split(X_test, y_test, test_size=0.5)


#Print split results
#print(f"Train shape: {X_train.shape[0]}")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"X_cv shape: {X_cv.shape}")

print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")
print(f"y_cv shape: {y_cv.shape}")


Train shape: 210
X_train shape: (210, 34)
X_test shape: (70, 34)
X_cv shape: (71, 34)
y_train shape: (210, 1)
y_test shape: (70, 1)
y_cv shape: (71, 1)


In [23]:
#Model creation

model = Sequential([
    tf.keras.Input(shape=(34,)),
    Dense(32, activation='relu', kernel_regularizer=l2(0.00)),
    Dense(32, activation='relu', kernel_regularizer=l2(0.00)),
    Dense(32, activation='relu', kernel_regularizer=l2(0.00)),
    Dense(1, activation='linear')
])

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                1120      
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 dense_2 (Dense)             (None, 32)                1056      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 3265 (12.75 KB)
Trainable params: 3265 (12.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['accuracy']    
)

#fit the model
history = model.fit(
    X, y,
    validation_data=(X_cv, y_cv),
    epochs=100
)

Epoch 1/100


UnimplementedError: Graph execution error:

Detected at node binary_crossentropy/Cast defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Python311\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Python311\Lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance

  File "C:\Python311\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start

  File "C:\Python311\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "C:\Python311\Lib\asyncio\base_events.py", line 607, in run_forever

  File "C:\Python311\Lib\asyncio\base_events.py", line 1919, in _run_once

  File "C:\Python311\Lib\asyncio\events.py", line 80, in _run

  File "C:\Python311\Lib\site-packages\ipykernel\kernelbase.py", line 516, in dispatch_queue

  File "C:\Python311\Lib\site-packages\ipykernel\kernelbase.py", line 505, in process_one

  File "C: